In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.output {font-size:10pt;}
div.input {font-family:Consolas; font-size:10pt; font-weight:bold;}
div.prompt {min-width:70px;}
</style>
"""))

In [3]:
import requests
from bs4 import BeautifulSoup

rss_url= 'https://rss.joins.com/joins_politics_list.xml'
money_response = requests.get(rss_url)
money_soup = BeautifulSoup(money_response.content, 'xml')

link_list = money_soup.select('item > link')
link_list = [l.text for l in link_list]

In [6]:
from konlpy.tag import Kkma

kkma = Kkma()
res = requests.get('https://news.joins.com/article/24029441?cloc=rss-news-politics')
soup = BeautifulSoup(res.content, 'html.parser')

article = soup.select_one('div#article_body').text
tagged_text = kkma.pos(article, 56)
nouns_list = [t[0] for t in tagged_text 
             if((t[1]=='NNG') | (t[1]=='NNP')) & 
             (len(t[0])>1) & (t[0]!='기자')]
print(nouns_list[:10])

['조치', '시행', '동안', '남북', '경협', '기업인', '고통', '해결', '법정', '오전']


In [11]:
from konlpy.tag import Kkma
kkma = Kkma()
news = []
for link in link_list:
    news_response = requests.get(link)
    news_soup = BeautifulSoup(news_response.content, "html.parser")
    element = news_soup.select_one('div#article_body')
    # 두가지 type이 있어서 나눔
    if element :
        article = element.text
    else:
        article = news_soup.select_one('p#leadCaption').text
        print(link, False)

    tagged_text = kkma.pos(article, 56)
    nouns_list = [t[0] for t in tagged_text if (t[1]=='NNG') & (len(t[0])>1) & (t[0]!='기자')] # 일반 명사만 추출
#   nouns_list = [t[0] for t in tagged_text if (t[1]=='NNP') & (len(t[0])>1) & (t[0]!='기자')] 고유명사만
    
    if len(nouns_list)>0 :
        news.append(nouns_list)
# len(news)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte

In [10]:
temp = []

for news_item in news:
    temp.append(' '.join(news_item))
    word_list = ' '.join(temp)

len(word_list), len(set(word_list))

(16691, 489)

In [12]:
import pandas as pd 
wordcount = pd.Series(dtype=np.int64)
words = word_list.split()

for word in words:
    if word in wordcount:
        wordcount[word] += 1
    else:
        wordcount[word] = 1

wordcount.sort_values(ascending=False, inplace=True)
freq_word = list(wordcount.head(10).index)
print(freq_word)

['의원', '백신', '지사', '국민', '대표', '민주당', '코로나', '정부', '이자', '접종']


In [13]:
temp = []

for news_item in news:
    t = [word for word in freq_word 
            if word in news_item]
    
    if len(t) > 0 :
        temp.append(t)
temp

[['의원', '대표', '민주당', '정부'],
 ['코로나', '정부'],
 ['의원', '지사', '국민', '대표', '민주당', '이자'],
 ['의원', '지사', '국민', '민주당', '정부'],
 ['의원', '백신', '국민', '민주당', '코로나', '정부', '이자', '접종'],
 ['의원', '지사', '국민', '대표', '민주당'],
 ['의원', '민주당', '코로나'],
 ['의원', '지사', '국민'],
 ['의원', '국민', '대표'],
 ['의원', '국민', '민주당', '정부'],
 ['국민', '코로나', '정부'],
 ['백신', '국민', '코로나', '정부', '이자', '접종'],
 ['의원', '국민', '대표'],
 ['백신', '코로나', '정부', '이자', '접종'],
 ['의원', '국민'],
 ['지사'],
 ['의원', '민주당'],
 ['의원', '대표', '민주당'],
 ['백신', '코로나', '정부', '접종'],
 ['의원', '지사'],
 ['의원', '국민', '대표', '민주당', '정부'],
 ['의원', '지사', '국민', '대표', '민주당', '이자']]

In [15]:
from apyori import apriori

rules = apriori(temp, 
                min_support=0.3, 
                min_confidence=0.2)
result = list(rules)
result

[RelationRecord(items=frozenset({'국민'}), support=0.5909090909090909, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'국민'}), confidence=0.5909090909090909, lift=1.0)]),
 RelationRecord(items=frozenset({'대표'}), support=0.36363636363636365, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'대표'}), confidence=0.36363636363636365, lift=1.0)]),
 RelationRecord(items=frozenset({'민주당'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'민주당'}), confidence=0.5, lift=1.0)]),
 RelationRecord(items=frozenset({'의원'}), support=0.7272727272727273, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'의원'}), confidence=0.7272727272727273, lift=1.0)]),
 RelationRecord(items=frozenset({'정부'}), support=0.45454545454545453, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'정부'}), confidence=0.45454545454545453, lift=1.0)]),
 Relation

In [17]:
for r in result:
    print(r, end='\n\n')

RelationRecord(items=frozenset({'국민'}), support=0.5909090909090909, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'국민'}), confidence=0.5909090909090909, lift=1.0)])

RelationRecord(items=frozenset({'대표'}), support=0.36363636363636365, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'대표'}), confidence=0.36363636363636365, lift=1.0)])

RelationRecord(items=frozenset({'민주당'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'민주당'}), confidence=0.5, lift=1.0)])

RelationRecord(items=frozenset({'의원'}), support=0.7272727272727273, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'의원'}), confidence=0.7272727272727273, lift=1.0)])

RelationRecord(items=frozenset({'정부'}), support=0.45454545454545453, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'정부'}), confidence=0.45454545454545453, lift=1.0)])

RelationRecord

In [ ]:
! jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
for r in result:
    print(r, end='\n\n')

In [ ]:
result[0]

In [ ]:
result[100]

In [ ]:
import pandas as pd 

result_df = pd.DataFrame(None, 
                        columns=['lhs', 'rhs', 
                                 'support', 'confidence', 'iift'])
index = 0

for row in result:
    support = row[1]
    ordered_st = row[2]
    
    for item in ordered_st:
        lhs = ','.join(x for x in item[0])
        rhs = ','.join(x.strip() for x in item[1])
        confidence = item[2]
        lift = item[3]
        if lift > 1:
            result_df.loc[index] = [lhs, rhs, 
                                    support, confidence, lift]
            index += 1
result_df